## Medical Text Classification using e-kNN 
where 'e' is epsilon & 'e' is the minimum similarity value requires to be in the nearest neighbour.

#### Importing the required libraries

In [1]:
#Importing the required libraries
import re
import math
import numpy as np
import pandas as pd
import string
import scipy as sp
import nltk
import time
import operator
from scipy import *
from scipy.sparse import *
from collections import defaultdict
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
#remove words less than length of 4
def filterLen(docs, minlen):
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]

#### Functions for building bag of words and csr_matrix ussing the bag of words

In [3]:
#Building Bag of words
def word_bag(docs):
#     nrows = len(docs)
    idx = {}
    tid = 0
    for d in docs:
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    return idx
    
#Building the sparse matrix
from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs, idx):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            #print(keys)
            if(k in idx):
                
                ind[j+n] = idx[k]
                val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

#### Functions to normalize the csr_matrix

In [4]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

#### e-kNN Function

In [5]:
#kNN Function
class knn_main():
    def __init__(self):
        pass
    
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train
    
    def predict(self, x_test,k,eps):
        self.x_test = x_test
        y_predict = []
        print(x_test.shape[0])
        for i,test_case in enumerate(x_test):
            temp = self.compute_distances(test_case, self.x_train)
            c = temp.tocoo(copy=True)
            d = self.sort_coo(c)
       #print(d[0])
            t = d[0:k]
        #print(t)
            dict_l ={}
        #print(k)
            for z,x in enumerate(t):
                index = x[1]
                similarity = x[2]
                label_t = self.y_train[index]
                if(z>0):
                    
                    if similarity > eps:
                        if label_t not in dict_l:
                            dict_l[label_t]=1
                        else:
                            dict_l[label_t]+=1
                else:
                    dict_l[label_t] =1
                
            m = max(dict_l.items(),key=operator.itemgetter(1))[0]
            y_predict.append(m)
            print("test case:", i+1,"Predicted :", m)
        return y_predict
    def sort_coo(self,m):
        tuples = zip(m.row, m.col, m.data)
        return sorted(tuples, key=lambda x: (x[2]),reverse=True)
    
    def compute_distances(self, X_test,train):        
        dot_pro = np.dot(X_test, train.T)
        return(dot_pro)

#### Reading the train data and storing it into list for performing classification

In [6]:
#Seperation of labels and text data
labels = []
texts = []
with open("train.dat", "r") as fh:
    train_lines = fh.readlines() 
for line in train_lines:
    splitline = line.split('\t')
    labels.append(splitline[0])
#     ps = word_tokenize()
    texts.append(nltk.word_tokenize(splitline[1].lower()))

len(texts)
# docs1 = filterLen(texts, 4)
# docs2 = filterLen(tex,4)

14438

#### Building word of bags for train data, building the csr_matrix and normalising it

In [7]:
train_text =  filterLen(texts, 4)
wordbag = word_bag(train_text)
train_mat = build_matrix(train_text,wordbag)
mat2 = csr_idf(train_mat, copy=True)
norm_train_mat = csr_l2normalize(mat2, copy=True)

#### Train test spilt the train data to find the evaluate the performance of our code and then build csr_matrix for test data using the wordbag of the train data

In [8]:
X_train, X_test, y_train,y_test = train_test_split(texts,labels,test_size = 0.3)
test_mat = build_matrix(X_test,wordbag)
mat3 = csr_idf(test_mat, copy=True)
norm_test_mat = csr_l2normalize(mat3, copy=True)

#### Calling the classifier function and performing predictions for the test split of train data

In [9]:
classifier = knn_main()
classifier.fit(norm_train_mat,labels)
g = classifier.predict(norm_test_mat,100,0.1)

4332
test case: 1 Predicted : 4
test case: 2 Predicted : 1
test case: 3 Predicted : 1
test case: 4 Predicted : 5
test case: 5 Predicted : 2
test case: 6 Predicted : 4
test case: 7 Predicted : 5
test case: 8 Predicted : 4
test case: 9 Predicted : 1
test case: 10 Predicted : 1
test case: 11 Predicted : 4
test case: 12 Predicted : 1
test case: 13 Predicted : 5
test case: 14 Predicted : 5
test case: 15 Predicted : 5
test case: 16 Predicted : 5
test case: 17 Predicted : 1
test case: 18 Predicted : 4
test case: 19 Predicted : 5
test case: 20 Predicted : 4
test case: 21 Predicted : 3
test case: 22 Predicted : 1
test case: 23 Predicted : 5
test case: 24 Predicted : 1
test case: 25 Predicted : 5
test case: 26 Predicted : 4
test case: 27 Predicted : 3
test case: 28 Predicted : 5
test case: 29 Predicted : 5
test case: 30 Predicted : 1
test case: 31 Predicted : 1
test case: 32 Predicted : 4
test case: 33 Predicted : 5
test case: 34 Predicted : 5
test case: 35 Predicted : 5
test case: 36 Predicted 

test case: 289 Predicted : 5
test case: 290 Predicted : 2
test case: 291 Predicted : 2
test case: 292 Predicted : 5
test case: 293 Predicted : 3
test case: 294 Predicted : 2
test case: 295 Predicted : 2
test case: 296 Predicted : 1
test case: 297 Predicted : 3
test case: 298 Predicted : 1
test case: 299 Predicted : 5
test case: 300 Predicted : 5
test case: 301 Predicted : 5
test case: 302 Predicted : 4
test case: 303 Predicted : 1
test case: 304 Predicted : 3
test case: 305 Predicted : 5
test case: 306 Predicted : 4
test case: 307 Predicted : 4
test case: 308 Predicted : 1
test case: 309 Predicted : 3
test case: 310 Predicted : 1
test case: 311 Predicted : 2
test case: 312 Predicted : 4
test case: 313 Predicted : 1
test case: 314 Predicted : 4
test case: 315 Predicted : 5
test case: 316 Predicted : 5
test case: 317 Predicted : 4
test case: 318 Predicted : 1
test case: 319 Predicted : 3
test case: 320 Predicted : 1
test case: 321 Predicted : 4
test case: 322 Predicted : 1
test case: 323

test case: 572 Predicted : 1
test case: 573 Predicted : 5
test case: 574 Predicted : 3
test case: 575 Predicted : 5
test case: 576 Predicted : 5
test case: 577 Predicted : 4
test case: 578 Predicted : 5
test case: 579 Predicted : 4
test case: 580 Predicted : 5
test case: 581 Predicted : 5
test case: 582 Predicted : 4
test case: 583 Predicted : 5
test case: 584 Predicted : 4
test case: 585 Predicted : 1
test case: 586 Predicted : 4
test case: 587 Predicted : 3
test case: 588 Predicted : 3
test case: 589 Predicted : 5
test case: 590 Predicted : 4
test case: 591 Predicted : 4
test case: 592 Predicted : 1
test case: 593 Predicted : 5
test case: 594 Predicted : 2
test case: 595 Predicted : 1
test case: 596 Predicted : 5
test case: 597 Predicted : 3
test case: 598 Predicted : 5
test case: 599 Predicted : 1
test case: 600 Predicted : 2
test case: 601 Predicted : 5
test case: 602 Predicted : 2
test case: 603 Predicted : 3
test case: 604 Predicted : 1
test case: 605 Predicted : 4
test case: 606

test case: 855 Predicted : 5
test case: 856 Predicted : 1
test case: 857 Predicted : 5
test case: 858 Predicted : 2
test case: 859 Predicted : 3
test case: 860 Predicted : 4
test case: 861 Predicted : 2
test case: 862 Predicted : 4
test case: 863 Predicted : 5
test case: 864 Predicted : 3
test case: 865 Predicted : 4
test case: 866 Predicted : 4
test case: 867 Predicted : 4
test case: 868 Predicted : 5
test case: 869 Predicted : 4
test case: 870 Predicted : 1
test case: 871 Predicted : 1
test case: 872 Predicted : 1
test case: 873 Predicted : 5
test case: 874 Predicted : 1
test case: 875 Predicted : 5
test case: 876 Predicted : 5
test case: 877 Predicted : 4
test case: 878 Predicted : 4
test case: 879 Predicted : 2
test case: 880 Predicted : 3
test case: 881 Predicted : 1
test case: 882 Predicted : 5
test case: 883 Predicted : 5
test case: 884 Predicted : 5
test case: 885 Predicted : 5
test case: 886 Predicted : 4
test case: 887 Predicted : 5
test case: 888 Predicted : 4
test case: 889

test case: 1134 Predicted : 2
test case: 1135 Predicted : 4
test case: 1136 Predicted : 5
test case: 1137 Predicted : 4
test case: 1138 Predicted : 1
test case: 1139 Predicted : 2
test case: 1140 Predicted : 1
test case: 1141 Predicted : 5
test case: 1142 Predicted : 4
test case: 1143 Predicted : 4
test case: 1144 Predicted : 1
test case: 1145 Predicted : 4
test case: 1146 Predicted : 3
test case: 1147 Predicted : 5
test case: 1148 Predicted : 5
test case: 1149 Predicted : 3
test case: 1150 Predicted : 5
test case: 1151 Predicted : 1
test case: 1152 Predicted : 1
test case: 1153 Predicted : 5
test case: 1154 Predicted : 4
test case: 1155 Predicted : 4
test case: 1156 Predicted : 3
test case: 1157 Predicted : 1
test case: 1158 Predicted : 5
test case: 1159 Predicted : 1
test case: 1160 Predicted : 5
test case: 1161 Predicted : 2
test case: 1162 Predicted : 5
test case: 1163 Predicted : 5
test case: 1164 Predicted : 5
test case: 1165 Predicted : 5
test case: 1166 Predicted : 2
test case:

...

test case: 3056 Predicted : 1
test case: 3057 Predicted : 1
test case: 3058 Predicted : 3
test case: 3059 Predicted : 5
test case: 3060 Predicted : 5
test case: 3061 Predicted : 2
test case: 3062 Predicted : 4
test case: 3063 Predicted : 5
test case: 3064 Predicted : 5
test case: 3065 Predicted : 3
test case: 3066 Predicted : 4
test case: 3067 Predicted : 4
test case: 3068 Predicted : 1
test case: 3069 Predicted : 5
test case: 3070 Predicted : 5
test case: 3071 Predicted : 5
test case: 3072 Predicted : 1
test case: 3073 Predicted : 4
test case: 3074 Predicted : 1
test case: 3075 Predicted : 1
test case: 3076 Predicted : 3
test case: 3077 Predicted : 1
test case: 3078 Predicted : 2
test case: 3079 Predicted : 5
test case: 3080 Predicted : 3
test case: 3081 Predicted : 5
test case: 3082 Predicted : 5
test case: 3083 Predicted : 4
test case: 3084 Predicted : 2
test case: 3085 Predicted : 3
test case: 3086 Predicted : 2
test case: 3087 Predicted : 5
test case: 3088 Predicted : 4
test case:

test case: 3332 Predicted : 3
test case: 3333 Predicted : 2
test case: 3334 Predicted : 4
test case: 3335 Predicted : 5
test case: 3336 Predicted : 3
test case: 3337 Predicted : 4
test case: 3338 Predicted : 4
test case: 3339 Predicted : 4
test case: 3340 Predicted : 3
test case: 3341 Predicted : 4
test case: 3342 Predicted : 5
test case: 3343 Predicted : 4
test case: 3344 Predicted : 4
test case: 3345 Predicted : 1
test case: 3346 Predicted : 1
test case: 3347 Predicted : 4
test case: 3348 Predicted : 4
test case: 3349 Predicted : 5
test case: 3350 Predicted : 1
test case: 3351 Predicted : 1
test case: 3352 Predicted : 4
test case: 3353 Predicted : 1
test case: 3354 Predicted : 1
test case: 3355 Predicted : 5
test case: 3356 Predicted : 1
test case: 3357 Predicted : 5
test case: 3358 Predicted : 2
test case: 3359 Predicted : 1
test case: 3360 Predicted : 5
test case: 3361 Predicted : 2
test case: 3362 Predicted : 5
test case: 3363 Predicted : 4
test case: 3364 Predicted : 5
test case:

test case: 3608 Predicted : 4
test case: 3609 Predicted : 4
test case: 3610 Predicted : 4
test case: 3611 Predicted : 5
test case: 3612 Predicted : 5
test case: 3613 Predicted : 2
test case: 3614 Predicted : 1
test case: 3615 Predicted : 1
test case: 3616 Predicted : 1
test case: 3617 Predicted : 1
test case: 3618 Predicted : 4
test case: 3619 Predicted : 1
test case: 3620 Predicted : 3
test case: 3621 Predicted : 3
test case: 3622 Predicted : 5
test case: 3623 Predicted : 3
test case: 3624 Predicted : 4
test case: 3625 Predicted : 4
test case: 3626 Predicted : 5
test case: 3627 Predicted : 3
test case: 3628 Predicted : 1
test case: 3629 Predicted : 5
test case: 3630 Predicted : 3
test case: 3631 Predicted : 5
test case: 3632 Predicted : 2
test case: 3633 Predicted : 5
test case: 3634 Predicted : 5
test case: 3635 Predicted : 1
test case: 3636 Predicted : 1
test case: 3637 Predicted : 2
test case: 3638 Predicted : 3
test case: 3639 Predicted : 4
test case: 3640 Predicted : 2
test case:

test case: 3883 Predicted : 2
test case: 3884 Predicted : 4
test case: 3885 Predicted : 4
test case: 3886 Predicted : 4
test case: 3887 Predicted : 3
test case: 3888 Predicted : 2
test case: 3889 Predicted : 3
test case: 3890 Predicted : 1
test case: 3891 Predicted : 3
test case: 3892 Predicted : 1
test case: 3893 Predicted : 1
test case: 3894 Predicted : 1
test case: 3895 Predicted : 4
test case: 3896 Predicted : 4
test case: 3897 Predicted : 3
test case: 3898 Predicted : 1
test case: 3899 Predicted : 4
test case: 3900 Predicted : 5
test case: 3901 Predicted : 1
test case: 3902 Predicted : 4
test case: 3903 Predicted : 3
test case: 3904 Predicted : 4
test case: 3905 Predicted : 4
test case: 3906 Predicted : 1
test case: 3907 Predicted : 2
test case: 3908 Predicted : 4
test case: 3909 Predicted : 5
test case: 3910 Predicted : 4
test case: 3911 Predicted : 1
test case: 3912 Predicted : 1
test case: 3913 Predicted : 5
test case: 3914 Predicted : 5
test case: 3915 Predicted : 5
test case:

test case: 4159 Predicted : 4
test case: 4160 Predicted : 3
test case: 4161 Predicted : 2
test case: 4162 Predicted : 5
test case: 4163 Predicted : 4
test case: 4164 Predicted : 1
test case: 4165 Predicted : 3
test case: 4166 Predicted : 5
test case: 4167 Predicted : 5
test case: 4168 Predicted : 3
test case: 4169 Predicted : 1
test case: 4170 Predicted : 1
test case: 4171 Predicted : 5
test case: 4172 Predicted : 1
test case: 4173 Predicted : 5
test case: 4174 Predicted : 4
test case: 4175 Predicted : 5
test case: 4176 Predicted : 4
test case: 4177 Predicted : 2
test case: 4178 Predicted : 2
test case: 4179 Predicted : 4
test case: 4180 Predicted : 5
test case: 4181 Predicted : 3
test case: 4182 Predicted : 5
test case: 4183 Predicted : 4
test case: 4184 Predicted : 4
test case: 4185 Predicted : 4
test case: 4186 Predicted : 1
test case: 4187 Predicted : 2
test case: 4188 Predicted : 4
test case: 4189 Predicted : 5
test case: 4190 Predicted : 5
test case: 4191 Predicted : 4
test case:

#### Building the confusion matrix and printing the performance report

In [10]:
cm=confusion_matrix(y_test ,g)
print(cm)
print(classification_report(y_test, g))

[[755  39  24  40 108]
 [ 49 256   5  25 114]
 [ 53  14 288  64 189]
 [ 27  14  16 754  94]
 [197 111  88 280 728]]
             precision    recall  f1-score   support

          1       0.70      0.78      0.74       966
          2       0.59      0.57      0.58       449
          3       0.68      0.47      0.56       608
          4       0.65      0.83      0.73       905
          5       0.59      0.52      0.55      1404

avg / total       0.64      0.64      0.63      4332



#### Now performing the above steps on the test data file

In [11]:
tst_tex = []
with open("test.dat", "r") as fr:
    test_lines = fr.readlines() 
for line in test_lines:
#         splitline = line.split()
    tst_tex.append(nltk.word_tokenize(line.lower()))
test_text = filterLen(tst_tex, 4)

In [12]:
test_mat = build_matrix(test_text,wordbag)
mat3 = csr_idf(test_mat, copy=True)
norm_test_mat = csr_l2normalize(mat3, copy=True)
# norm_test_mat

In [13]:
classifier = knn_main()
classifier.fit(norm_train_mat,labels)
#print(mat5)

In [ ]:
g = classifier.predict(norm_test_mat,100,0.1)

14442
test case: 1 Predicted : 5
test case: 2 Predicted : 5
test case: 3 Predicted : 5
test case: 4 Predicted : 2
test case: 5 Predicted : 1
test case: 6 Predicted : 3
test case: 7 Predicted : 3
test case: 8 Predicted : 4
test case: 9 Predicted : 1
test case: 10 Predicted : 5
test case: 11 Predicted : 5
test case: 12 Predicted : 4
test case: 13 Predicted : 4
test case: 14 Predicted : 5
test case: 15 Predicted : 5
test case: 16 Predicted : 1
test case: 17 Predicted : 5
test case: 18 Predicted : 1
test case: 19 Predicted : 4
test case: 20 Predicted : 4
test case: 21 Predicted : 3
test case: 22 Predicted : 1
test case: 23 Predicted : 5
test case: 24 Predicted : 4
test case: 25 Predicted : 1
test case: 26 Predicted : 5
test case: 27 Predicted : 4
test case: 28 Predicted : 4
test case: 29 Predicted : 5
test case: 30 Predicted : 1
test case: 31 Predicted : 3
test case: 32 Predicted : 5
test case: 33 Predicted : 2
test case: 34 Predicted : 1
test case: 35 Predicted : 4
test case: 36 Predicted

test case: 290 Predicted : 5
test case: 291 Predicted : 1
test case: 292 Predicted : 3
test case: 293 Predicted : 5
test case: 294 Predicted : 1
test case: 295 Predicted : 5
test case: 296 Predicted : 4
test case: 297 Predicted : 5
test case: 298 Predicted : 5
test case: 299 Predicted : 5
test case: 300 Predicted : 5
test case: 301 Predicted : 4
test case: 302 Predicted : 4
test case: 303 Predicted : 5
test case: 304 Predicted : 1
test case: 305 Predicted : 1
test case: 306 Predicted : 1
test case: 307 Predicted : 1
test case: 308 Predicted : 4
test case: 309 Predicted : 5
test case: 310 Predicted : 5
test case: 311 Predicted : 5
test case: 312 Predicted : 1
test case: 313 Predicted : 2
test case: 314 Predicted : 1
test case: 315 Predicted : 4
test case: 316 Predicted : 4
test case: 317 Predicted : 5
test case: 318 Predicted : 5
test case: 319 Predicted : 5
test case: 320 Predicted : 3
test case: 321 Predicted : 2
test case: 322 Predicted : 5
test case: 323 Predicted : 3
test case: 324

test case: 574 Predicted : 5
test case: 575 Predicted : 1
test case: 576 Predicted : 2
test case: 577 Predicted : 1
test case: 578 Predicted : 4
test case: 579 Predicted : 5
test case: 580 Predicted : 4
test case: 581 Predicted : 5
test case: 582 Predicted : 4
test case: 583 Predicted : 1
test case: 584 Predicted : 3
test case: 585 Predicted : 2
test case: 586 Predicted : 4
test case: 587 Predicted : 5
test case: 588 Predicted : 4
test case: 589 Predicted : 1
test case: 590 Predicted : 5
test case: 591 Predicted : 3
test case: 592 Predicted : 5
test case: 593 Predicted : 1
test case: 594 Predicted : 4
test case: 595 Predicted : 3
test case: 596 Predicted : 5
test case: 597 Predicted : 5
test case: 598 Predicted : 5
test case: 599 Predicted : 1
test case: 600 Predicted : 3
test case: 601 Predicted : 3
test case: 602 Predicted : 4
test case: 603 Predicted : 1
test case: 604 Predicted : 4
test case: 605 Predicted : 5
test case: 606 Predicted : 1
test case: 607 Predicted : 3
test case: 608

test case: 857 Predicted : 3
test case: 858 Predicted : 2
test case: 859 Predicted : 5
test case: 860 Predicted : 3
test case: 861 Predicted : 4
test case: 862 Predicted : 1
test case: 863 Predicted : 5
test case: 864 Predicted : 1
test case: 865 Predicted : 4
test case: 866 Predicted : 2
test case: 867 Predicted : 5
test case: 868 Predicted : 1
test case: 869 Predicted : 5
test case: 870 Predicted : 2
test case: 871 Predicted : 4
test case: 872 Predicted : 3
test case: 873 Predicted : 1
test case: 874 Predicted : 4
test case: 875 Predicted : 4
test case: 876 Predicted : 5
test case: 877 Predicted : 4
test case: 878 Predicted : 5
test case: 879 Predicted : 4
test case: 880 Predicted : 4
test case: 881 Predicted : 4
test case: 882 Predicted : 4
test case: 883 Predicted : 4
test case: 884 Predicted : 1
test case: 885 Predicted : 4
test case: 886 Predicted : 5
test case: 887 Predicted : 3
test case: 888 Predicted : 1
test case: 889 Predicted : 2
test case: 890 Predicted : 5
test case: 891

test case: 1135 Predicted : 5
test case: 1136 Predicted : 4
test case: 1137 Predicted : 1
test case: 1138 Predicted : 4
test case: 1139 Predicted : 5
test case: 1140 Predicted : 4
test case: 1141 Predicted : 1
test case: 1142 Predicted : 1
test case: 1143 Predicted : 5
test case: 1144 Predicted : 3
test case: 1145 Predicted : 5
test case: 1146 Predicted : 3
test case: 1147 Predicted : 4
test case: 1148 Predicted : 1
test case: 1149 Predicted : 3
test case: 1150 Predicted : 2
test case: 1151 Predicted : 4
test case: 1152 Predicted : 1
test case: 1153 Predicted : 5
test case: 1154 Predicted : 1
test case: 1155 Predicted : 5
test case: 1156 Predicted : 3
test case: 1157 Predicted : 5
test case: 1158 Predicted : 4
test case: 1159 Predicted : 5
test case: 1160 Predicted : 4
test case: 1161 Predicted : 5
test case: 1162 Predicted : 5
test case: 1163 Predicted : 4
test case: 1164 Predicted : 5
test case: 1165 Predicted : 4
test case: 1166 Predicted : 5
test case: 1167 Predicted : 4
test case:

...

test case: 7452 Predicted : 5
test case: 7453 Predicted : 1
test case: 7454 Predicted : 1
test case: 7455 Predicted : 3
test case: 7456 Predicted : 4
test case: 7457 Predicted : 5
test case: 7458 Predicted : 3
test case: 7459 Predicted : 5
test case: 7460 Predicted : 1
test case: 7461 Predicted : 5
test case: 7462 Predicted : 4
test case: 7463 Predicted : 4
test case: 7464 Predicted : 5
test case: 7465 Predicted : 4
test case: 7466 Predicted : 5
test case: 7467 Predicted : 1
test case: 7468 Predicted : 5
test case: 7469 Predicted : 2
test case: 7470 Predicted : 1
test case: 7471 Predicted : 1
test case: 7472 Predicted : 4
test case: 7473 Predicted : 1
test case: 7474 Predicted : 1
test case: 7475 Predicted : 4
test case: 7476 Predicted : 5
test case: 7477 Predicted : 3
test case: 7478 Predicted : 3
test case: 7479 Predicted : 5
test case: 7480 Predicted : 5
test case: 7481 Predicted : 5
test case: 7482 Predicted : 5
test case: 7483 Predicted : 1
test case: 7484 Predicted : 3
test case:

test case: 7728 Predicted : 4
test case: 7729 Predicted : 5
test case: 7730 Predicted : 5
test case: 7731 Predicted : 5
test case: 7732 Predicted : 1
test case: 7733 Predicted : 2
test case: 7734 Predicted : 4
test case: 7735 Predicted : 3
test case: 7736 Predicted : 3
test case: 7737 Predicted : 5
test case: 7738 Predicted : 3
test case: 7739 Predicted : 1
test case: 7740 Predicted : 4
test case: 7741 Predicted : 5
test case: 7742 Predicted : 1
test case: 7743 Predicted : 5
test case: 7744 Predicted : 5
test case: 7745 Predicted : 5
test case: 7746 Predicted : 2
test case: 7747 Predicted : 2
test case: 7748 Predicted : 4
test case: 7749 Predicted : 1
test case: 7750 Predicted : 5
test case: 7751 Predicted : 4
test case: 7752 Predicted : 5
test case: 7753 Predicted : 2
test case: 7754 Predicted : 1
test case: 7755 Predicted : 5
test case: 7756 Predicted : 4
test case: 7757 Predicted : 4
test case: 7758 Predicted : 5
test case: 7759 Predicted : 5
test case: 7760 Predicted : 5
test case:

test case: 8004 Predicted : 1
test case: 8005 Predicted : 1
test case: 8006 Predicted : 5
test case: 8007 Predicted : 3
test case: 8008 Predicted : 5
test case: 8009 Predicted : 1
test case: 8010 Predicted : 5
test case: 8011 Predicted : 4
test case: 8012 Predicted : 4
test case: 8013 Predicted : 5
test case: 8014 Predicted : 4
test case: 8015 Predicted : 5
test case: 8016 Predicted : 5
test case: 8017 Predicted : 1
test case: 8018 Predicted : 1
test case: 8019 Predicted : 4
test case: 8020 Predicted : 2
test case: 8021 Predicted : 1
test case: 8022 Predicted : 4
test case: 8023 Predicted : 1
test case: 8024 Predicted : 2
test case: 8025 Predicted : 3
test case: 8026 Predicted : 4
test case: 8027 Predicted : 5
test case: 8028 Predicted : 5
test case: 8029 Predicted : 2
test case: 8030 Predicted : 5
test case: 8031 Predicted : 1
test case: 8032 Predicted : 1
test case: 8033 Predicted : 5
test case: 8034 Predicted : 2
test case: 8035 Predicted : 2
test case: 8036 Predicted : 1
test case:

test case: 8280 Predicted : 5
test case: 8281 Predicted : 5
test case: 8282 Predicted : 5
test case: 8283 Predicted : 4
test case: 8284 Predicted : 5
test case: 8285 Predicted : 5
test case: 8286 Predicted : 1
test case: 8287 Predicted : 5
test case: 8288 Predicted : 1
test case: 8289 Predicted : 1
test case: 8290 Predicted : 4
test case: 8291 Predicted : 5
test case: 8292 Predicted : 4
test case: 8293 Predicted : 1
test case: 8294 Predicted : 1
test case: 8295 Predicted : 4
test case: 8296 Predicted : 2
test case: 8297 Predicted : 5
test case: 8298 Predicted : 1
test case: 8299 Predicted : 5
test case: 8300 Predicted : 5
test case: 8301 Predicted : 1
test case: 8302 Predicted : 5
test case: 8303 Predicted : 3
test case: 8304 Predicted : 5
test case: 8305 Predicted : 5
test case: 8306 Predicted : 5
test case: 8307 Predicted : 4
test case: 8308 Predicted : 4
test case: 8309 Predicted : 2
test case: 8310 Predicted : 1
test case: 8311 Predicted : 5
test case: 8312 Predicted : 2
test case:

test case: 8556 Predicted : 4
test case: 8557 Predicted : 1
test case: 8558 Predicted : 4
test case: 8559 Predicted : 4
test case: 8560 Predicted : 3
test case: 8561 Predicted : 5
test case: 8562 Predicted : 5
test case: 8563 Predicted : 5
test case: 8564 Predicted : 5
test case: 8565 Predicted : 2
test case: 8566 Predicted : 3
test case: 8567 Predicted : 1
test case: 8568 Predicted : 4
test case: 8569 Predicted : 5
test case: 8570 Predicted : 5
test case: 8571 Predicted : 4
test case: 8572 Predicted : 5
test case: 8573 Predicted : 1
test case: 8574 Predicted : 3
test case: 8575 Predicted : 1
test case: 8576 Predicted : 4
test case: 8577 Predicted : 1
test case: 8578 Predicted : 4
test case: 8579 Predicted : 1
test case: 8580 Predicted : 4
test case: 8581 Predicted : 1
test case: 8582 Predicted : 5
test case: 8583 Predicted : 2
test case: 8584 Predicted : 2
test case: 8585 Predicted : 5
test case: 8586 Predicted : 4
test case: 8587 Predicted : 1
test case: 8588 Predicted : 4
test case:

#### Writing the predicted class labels into a file

In [ ]:
with open('program.dat', 'w') as f:
        for cls in g:
            f.write("%s\n" % cls)